In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import re
import numpy as np
import pandas as pd
import pymysql

In [33]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

In [11]:
def createtableBrands():
    try:
        conn = pymysql.connect('localhost', 'msba', 'hku2019', 'makeup')
        conn.ping(False)
        cur = conn.cursor()
        cur.execute('DROP TABLE IF EXISTS brands')
        cur.execute('CREATE TABLE brands (id INT UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY, brandCN VARCHAR(80) DEFAULT NULL, brandEN VARCHAR(80) DEFAULT NULL)')
        conn.commit()
    except:
        conn.rollback()
    finally:
        if conn:
            conn.close()
        if cur:
            cur.close()

def connMySQL():
    conn = pymysql.connect('localhost', 'msba', 'hku2019', 'makeup')
    conn.autocommit(1)
    cur = conn.cursor()
    return cur

# Web Spider

In [5]:
data = []

driver = webdriver.Firefox(executable_path = '/usr/local/bin/geckodriver')
driver.get("https://cosme.pclady.com.cn/brand_list.html")
driver.implicitly_wait(5)

time_start = time.time()

for char in tqdm(range(26)):
    
    index = str(6+char)
    xpath1 ='//*[@id="J-Select"]/div['+index+']/div[2]/ul'
    content1 = driver.find_elements_by_xpath(xpath1)[0].text
    xpath2 = '//*[@id="J-Select"]/div['+index+']/div[2]/ul/li'
    content2_list = driver.find_elements_by_xpath(xpath2)
    
    if content1 == "":
        content2 = [[a.find_elements_by_css_selector('span')[0].text,a.find_elements_by_css_selector('span')[1].text] 
                    for a in content2_list]
    else:
        content2_1 = [a1.text.split(" ",1) for a1 in content2_list[0:5]]
        content2_2 = [[a.find_elements_by_css_selector('span')[0].text,a.find_elements_by_css_selector('span')[1].text] 
                      for a in content2_list[5:]]
        content2 = content2_1 + content2_2
        
    data += content2

time_end = time.time()

print('Time:', time_end - time_start, 's') 

100%|██████████| 26/26 [03:26<00:00, 12.58s/it]

Time: 206.77502703666687 s


In [7]:
data

[['安娜苏', 'Anna Sui'],
 ['奥蜜思', 'ORBIS'],
 ['阿芙', 'AFU'],
 ['伊蒂之屋', 'ETUDE HOUSE'],
 ['艾诗缇', 'ASTALIFT'],
 ['嫒碧知', 'episteme'],
 ['嗳呵', 'ELSKER'],
 ['瑷露德玛', 'aloe derma'],
 ['嫒色', 'est'],
 ['奥琪', ''],
 ['艾芙', 'Efasit'],
 ['安婷', 'ANTIN'],
 ['阿芙', 'AFU'],
 ['安婕妤', 'Anglee'],
 ['艾黛儿', 'Ardell'],
 ['艾妃儿', 'Aiffil'],
 ['阿皙尔', 'ASSUR'],
 ['奥黛拉', 'AUDALA'],
 ['安芙兰', ''],
 ['爱茉莉', 'Amore'],
 ['爱茉莉太平洋', 'AMOREPACIFIC'],
 ['艾芙美', 'Aderma'],
 ['爱琪美', 'Academie'],
 ['奥岚雪', 'OVID'],
 ['澳芙园', 'Aufly'],
 ['阿缇雅香集', 'Ateya'],
 ['安安', 'anan'],
 ['Aa Kode+', ''],
 ['安安国际', ''],
 ['澳宝', ''],
 ['艾博妍', 'Erborian'],
 ['奥碧虹', 'Albion'],
 ['爱泊丽', 'ASPLIR'],
 ['奥泊丽有机', 'Aubrey Organics'],
 ['艾伯特尼波', 'Albert Nipon'],
 ['傲蝶', 'Ottie'],
 ['艾的', 'IDA'],
 ['阿迪达斯', 'Adidas'],
 ['艾迪儿', 'Idee'],
 ['爱德哈代', 'Ed Hardy'],
 ['安多可', 'ENDOCARE'],
 ['安东尼男士', 'Anthony Logistics for Men'],
 ['奥迪氏', 'Euto-Altas'],
 ['艾杜纱', 'Ettusais'],
 ['爱迪威', 'adwin'],
 ['阿蒂仙之香', 'L‘artisan Parfumeur'],
 ['埃儿贝夫', 'Airbeutf '],
 ['阿尔法', 'alpha'],

In [8]:
len(data)

3494

# Data Preprocessing

## Normalization

In [47]:
CNpattern = re.compile(u'[\u4e00-\u9fa5]+')
NCNpattern = re.compile(u'[^\u4e00-\u9fa5]+')

brands = []

for row in data:
    
    if len(row) == 2 and row[1] != '':
        
        if CNpattern.search(row[0]):
            
            if row[1] in row[0]:
                bdnameCN = CNpattern.findall(row[0])[0].strip()
                bdnameEN = NCNpattern.findall(row[0])[0].strip()
            
            else:
                bdnameCN = row[0]
                bdnameEN = row[1]

        else:
            bdnameCN = 'NA'
        
            if row[0][:3].lower() == row[1][:3].lower():
                if len(row[0]) > len(row[1]):
                    bdnameEN = row[0]
                else:
                    bdnameEN = row[1]
            else:
                bdnameEN = row[0] + ' '+ row[1]
    
    else:
        
        if all(map(lambda i:'\u4e00' <= i <= '\u9fa5', row[0])):
            bdnameCN = row[0]
            bdnameEN = 'NA'
        
        elif CNpattern.search(row[0]):
            bdnameCN = CNpattern.findall(row[0])[0].strip()
            bdnameEN = NCNpattern.findall(row[0])[0].strip()
        else:
            bdnameCN = 'NA'
            bdnameEN = row[0]
    
       
    tmp = [bdnameCN, bdnameEN]
    brands.append(tmp)

In [10]:
brands # 3494

[['安娜苏', 'Anna Sui'],
 ['奥蜜思', 'ORBIS'],
 ['阿芙', 'AFU'],
 ['伊蒂之屋', 'ETUDE HOUSE'],
 ['艾诗缇', 'ASTALIFT'],
 ['嫒碧知', 'episteme'],
 ['嗳呵', 'ELSKER'],
 ['瑷露德玛', 'aloe derma'],
 ['嫒色', 'est'],
 ['奥琪', 'NA'],
 ['艾芙', 'Efasit'],
 ['安婷', 'ANTIN'],
 ['阿芙', 'AFU'],
 ['安婕妤', 'Anglee'],
 ['艾黛儿', 'Ardell'],
 ['艾妃儿', 'Aiffil'],
 ['阿皙尔', 'ASSUR'],
 ['奥黛拉', 'AUDALA'],
 ['安芙兰', 'NA'],
 ['爱茉莉', 'Amore'],
 ['爱茉莉太平洋', 'AMOREPACIFIC'],
 ['艾芙美', 'Aderma'],
 ['爱琪美', 'Academie'],
 ['奥岚雪', 'OVID'],
 ['澳芙园', 'Aufly'],
 ['阿缇雅香集', 'Ateya'],
 ['安安', 'anan'],
 ['NA', 'Aa Kode+'],
 ['安安国际', 'NA'],
 ['澳宝', 'NA'],
 ['艾博妍', 'Erborian'],
 ['奥碧虹', 'Albion'],
 ['爱泊丽', 'ASPLIR'],
 ['奥泊丽有机', 'Aubrey Organics'],
 ['艾伯特尼波', 'Albert Nipon'],
 ['傲蝶', 'Ottie'],
 ['艾的', 'IDA'],
 ['阿迪达斯', 'Adidas'],
 ['艾迪儿', 'Idee'],
 ['爱德哈代', 'Ed Hardy'],
 ['安多可', 'ENDOCARE'],
 ['安东尼男士', 'Anthony Logistics for Men'],
 ['奥迪氏', 'Euto-Altas'],
 ['艾杜纱', 'Ettusais'],
 ['爱迪威', 'adwin'],
 ['阿蒂仙之香', 'L‘artisan Parfumeur'],
 ['埃儿贝夫', 'Airbeutf '],
 ['阿尔法',

## Remove Duplicates

In [49]:
# remove duplicated brands: 3384 left
CN = []
EN = []
for i in range(len(brands)):
    CN.append(brands[i][0])
    EN.append(brands[i][1])
brandsdict = {'brandCN': CN, 'brandEN': EN}
brandsdf = pd.DataFrame(brandsdict)

brandsdf = brandsdf.drop_duplicates()
brandsdf.head()
len(brandsdf)
brandsdf.to_csv("3384brands.csv", header = None, encoding = 'utf_8_sig')

,brandCN,brandEN
0,安娜苏,Anna Sui
1,奥蜜思,ORBIS
2,阿芙,AFU
3,伊蒂之屋,ETUDE HOUSE
4,艾诗缇,ASTALIFT


3384

## Append Brands

In [27]:
brandsdf = pd.read_csv("3384brands.csv", keep_default_na = False)
brandsdf.columns = ['Index', 'brandCN', 'brandEN']
brandsdf = brandsdf[['brandCN', 'brandEN']]
brandsdf

,brandCN,brandEN
0,奥蜜思,ORBIS
1,阿芙,AFU
2,伊蒂之屋,ETUDE HOUSE
3,艾诗缇,ASTALIFT
4,嫒碧知,episteme
5,嗳呵,ELSKER
6,瑷露德玛,aloe derma
7,嫒色,est
8,奥琪,NA
9,艾芙,Efasit


In [31]:
append_brands = pd.read_excel("50brands.xlsx", keep_default_na = False, encoding = 'utf-8', header = None)
append_brands.columns = ['brandCN', 'brandEN'] 
append_brands

,brandCN,brandEN
0,三生花,SANSEN
1,橘朵,Judydoll
2,麦吉丽,Mageline
3,寻荟记,ALL ABOUT ALOE
4,粉瑟,FUNSACE
5,葵儿,CUIR
6,健美创研,May Greate
7,丽普司肽,LIPOSTIDES
8,迷奇,Miracle
9,优时颜,UNISKIN


In [41]:
append_brands = pd.read_excel("50brands.xlsx", keep_default_na = False, encoding = 'utf-8', header = None)
append_brands.columns = ['brandCN', 'brandEN'] 
combine_brands = brandsdf.append(append_brands).drop_duplicates()
combine_brands = combine_brands.reset_index(drop = True)

combine_brands.tail()
len(combine_brands)
combine_brands.to_csv("3432brands.csv", encoding = 'utf_8_sig')

,brandCN,brandEN
3427,乔菲特,QIAOFEITE
3428,优斐斯,UFACE
3429,斐思妮,FACE LIVE
3430,艺霏,Easyface
3431,雅莎,YASHA


3432

# MySQL Connection

In [12]:
createtableBrands()

/Applications/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1051, "Unknown table 'makeup.brands'")
  result = self._query(query)


In [13]:
cur = connMySQL()
for brand in brands:
    cur.execute('INSERT INTO brands (brandCN, brandEN) VALUES (%s, %s)', (brand[0], brand[1]))

1

1

1

1

1

...

1

1

1

1

1

In [14]:
sqldldupli = '''DELETE FROM brands WHERE (brandCN,brandEN) IN (
   SELECT t1.brandCN,t1.brandEN FROM (
   SELECT brandCN,brandEN FROM brands GROUP BY brandCN,brandEN HAVING count(1)> 1) AS t1) AND id NOT IN (
   SELECT t2.minid FROM (
   SELECT min(id) AS minid FROM brands GROUP BY brandCN,brandEN HAVING count(1)> 1) t2)'''

In [15]:
cur.execute(sqldldupli)

110

In [16]:
cur.close()